In [1]:
import os
import json
import numpy as np
# import robotic as ry
from tqdm import tqdm
from lookup import features, contraint_types
from komo_generators import generate_random_komo, generate_random_pick
# from random_kitchen_generator import generate_random_kitchen

In [2]:
# C = generate_random_kitchen(seed=84979, root_path="/home/denis/rai-robotModels/robocasa/")
# frame_names = C.getFrameNames()
# features["qItself"][1] = len(C.getJointState())
frame_names = ["r_gripper", "omnibase", "floor", "obj1", "obj2"]

## Generate KOMO texts

In [3]:
# Special Features (for pure random KOMOs)
lucky_frames = ["r_gripper", "r_palm"]
lucky_features = ["positionDiff", "position", "vectorX", "vectorY", "vectorZ", "quaternion", "negDistance"]

# Special Features (for random manipulation KOMOs)
end_effector_frames = ["r_gripper"]
mobile_base_frames = ["omnibase"]
object_frames = []  # Frames that are not attached to anything that can be manipulated with basic pick, place, push, etc.
jointed_frames = []  # Frames that have a parent / grandparent / great-grandparent / etc. that is a joint (Not including robot)
world_attach_frames = ["floor"]  # Like table in manipulation.py, some sort of world frame that tells us the world rotation and for us to place stuff after a mode switch.

In [4]:
# komo_generator = lambda: generate_random_komo(frame_names, features, lucky_frames, lucky_features)
komo_generator = lambda: generate_random_pick(end_effector_frames, object_frames, world_attach_frames)

In [ ]:
for i in range(10):
    komo_text = komo_generator()
    print(komo_text)

In [ ]:
komo_samples = 10

outfolder = "./komo_samples"
text_outfolder = os.path.join(outfolder, "texts")
if not os.path.exists(text_outfolder):
    os.makedirs(text_outfolder)
else:
    for file in os.listdir(text_outfolder):
        file_path = os.path.join(text_outfolder, file)
        os.remove(file_path)

for i in tqdm(range(komo_samples)):
    komo_text = generate_random_komo()
    filename = os.path.join(text_outfolder, f"komo{i}.txt")
    with open(filename, "w") as file:
        file.write(komo_text)

## Classify KOMO texts

In [ ]:
total_komos = 0
total_feasible_komos = 0
total_error_komos = 0

feasible_komos = []
error_komos = []

for i in tqdm(range(komo_samples)):
    
    filename = os.path.join(text_outfolder, f"komo{i}.txt")
    with open(filename, "r") as file:
        komo_text = file.read()

    # print(f"Processing file {filename}...")
    total_komos += 1
    
    try:
        komo = ry.KOMO(C, 1, 1, 0, False)
        exec(komo_text)
        ret = ry.NLP_Solver(komo.nlp(), verbose=0).solve()
    except:
        total_error_komos += 1
        error_komos.append(i)
        continue
    
    if ret.feasible:
        total_feasible_komos += 1
        feasible_komos.append(i)
    
    if (i+1)%(komo_samples//10) == 0:
        print(f"Total feasible komos: {total_feasible_komos}")
        print(f"Total error komos: {total_error_komos}")
        print(f"Total komos: {total_komos}")
        print(f"{(total_feasible_komos/total_komos)*100:.2f}% feasible")

In [ ]:
result = f"""Total feasible komos - Total error komos - Total komos: 
{total_feasible_komos} - {total_error_komos} - {total_komos}. 
({(total_feasible_komos/total_komos)*100:.2f}% feasible)"""
print(result)
print(feasible_komos)
results_file_path = os.path.join(outfolder, "feasibles.json")
results_dict = {
    "feasible_komos": feasible_komos,
    "error_komos": error_komos,
}
json.dump(results_dict, open(results_file_path, 'w'))

## Visualize feasible KOMOs

In [ ]:
feasibe_samples_count = 10
feasibe_samples = np.random.choice(feasible_komos, feasibe_samples_count, replace=False)

for i in feasibe_samples:

    filename = os.path.join(text_outfolder, f"komo{i}.txt")
    with open(filename, "r") as file:
        komo_text = file.read()
    print(f"__________ SAMPLE {i} __________")
    print(komo_text)

    komo = ry.KOMO(C, 1, 1, 0, False)
    exec(komo_text)
    ret = ry.NLP_Solver(komo.nlp(), verbose=0).solve()

    komo.view_play(True)